In [196]:
import pandas as pd
import io
from io import BytesIO
from io import StringIO
from datetime import date
from datetime import datetime,timedelta
import ibm_boto3
from ibm_botocore.client import Config
import shutil
try:
    !pip install paramiko
    import paramiko
except:
    import paramiko, StringIO
    
try:
    ! pip install pgpy
    import pgpy
except:
    import pgpy
import os
from io import BytesIO
import datetime
import pgpy

In [197]:

# @hidden_cell
# The following code contains metadata for connection "WCO_Connection" in your project.
# You might want to remove secret properties before you share your notebook.

WCO_Connection_metadata = {
    "key_passphrase": """L2Csftp@frankfurt""",
    "auth_method": """username_password_key""",
    "password": """L2Csftp@frankfurt""",
    "port": """22""",
    "connection_mode": """sftp""",
    "host": """sftp-gw.gpsemea.ihost.com""",
    "private_key": """--------END RSA PRIVATE KEY-----
""",
    "username": """ibmkrkana001"""
}

# Create connection to SFTP server

In [198]:
username = 'ibmkrkana001'
password = 'L2Csftp@frankfurt'   # win scpt
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
private_key = StringIO(WCO_Connection_metadata['private_key'])
pk = paramiko.RSAKey.from_private_key(private_key,password=WCO_Connection_metadata['password'])
ssh.connect(hostname=WCO_Connection_metadata['host'],username=username,password=WCO_Connection_metadata['password'],
           pkey=pk)
connection = ssh.open_sftp()

In [199]:
rmtPth = '/ANALYTICS/WCO/'
current_directory = os.getcwd()
locPth = current_directory + '/'

PGP Key

In [200]:
PGP_Key_data="""-----BEND PGP PRIVATE KEY BLOCK-----
"""

In [201]:
PGP_Key1 = StringIO(PGP_Key_data)
pasPhrase="L2Cpgp@analytics"

# check folder structure on local

In [202]:
def LocalCheckOrCreateFolder(locPth): #check local folder if missing create
    dirList = ['Ar_Archive', 'Ar_Decrypted']
    for di in dirList:
        dirpath = locPth + di
      
        if os.path.exists(dirpath):
            shutil.rmtree(dirpath)
            os.makedirs(dirpath)
        else:
            os.makedirs(dirpath)
    print("All folder structure available at Local ")

In [203]:
LocalCheckOrCreateFolder(locPth)

All folder structure available at Local 


In [204]:
# for file in os.listdir(locPth):
#             print(file)

# Check folder structure on Remote 


In [205]:
def RMTcheckOrCreateFolder(rmtPth): # check remote folder if folder missing create
    dirList = ['Ar_Archive','Ar_Decrypted','Ar_RAW']
    for di in dirList:
        dirpath = rmtPth + di
        try:
            connection.chdir(dirpath)
        except IOError:
            connection.mkdir(dirpath)
            connectionchmod(dirpath, mode='0o777')
    return "All file folder Available"

In [206]:
RMTcheckOrCreateFolder(rmtPth)

'All file folder Available'

# Decryption function

In [207]:
def decryptFile(fil):  # function to decrypt file
    if fil[-8:] == '.txt.gpg':
        key,_ = pgpy.PGPKey.from_blob(PGP_Key_data)
#         key, _ = pgpy.PGPKey.from_file(PGP_Key)
        emsg = pgpy.PGPMessage.from_file(fil)
        with key.unlock(pasPhrase):
            d = key.decrypt(emsg).message
            try:
                df = pd.read_csv(BytesIO(d), error_bad_lines=False, sep='|')
            except:
                df = pd.read_csv(BytesIO(d), error_bad_lines=False, sep='|', encoding='cp1252')
        return df
    


# validate date and day fucntion

In [208]:
def checkSunday(file, nDays, previousDay):  # decrypt if file received on Sunday
    Current_Date = datetime.datetime.today()
    Previous_Date = datetime.datetime.today() - datetime.timedelta(days=nDays)  # days = 1 for AP 2 for AR if running it on Monday
    Previous_Date_7 = datetime.datetime.today() - datetime.timedelta(days=previousDay)  # days = 7 if running it on Monday
    previsusDay = Previous_Date.strftime('%A')
    try:
        strdate = file.split('_')[2][:8]
        m = int(strdate[:2])
        d = int(strdate[2:4])
        y = int(strdate[4:])
        filedate = datetime.datetime(y, m, d)
        fileDay = filedate.strftime('%A')
    except:
        strdate = file.split('_')[3][:8]
        m = int(strdate[:2])
        d = int(strdate[2:4])
        y = int(strdate[4:])
        filedate = datetime.datetime(y, m, d)
        fileDay = filedate.strftime('%A')
    return filedate, fileDay, previsusDay, Previous_Date, Previous_Date_7

# Download AR data from sftp server into temp path

In [209]:
    def download_AR(rmtPth, locPth,nDays,previousDay):      #download AR data
        nDays = nDays    # if running it on Monday it will download the Saturday's file
        previousDay = previousDay
        listDownloadFileAR = []
        for file in connection.listdir(rmtPth):
            typeFile = ['ARClosedItemDaily', 'AROpenItemDaily', 'CustMstrDaily', 'ARDisputesDaily']
            filedate, fileDay, previsusDay, Previous_Date, Previous_Date_7  = checkSunday(file, nDays,previousDay)

            splitFileType=file.split('_')[1]
            # if filedate.date() == Previous_Date.date() and previsusDay == 'Saturday' and splitFileType in typeFile:
            if filedate.date() == Previous_Date.date() and splitFileType in typeFile:
                listDownloadFileAR.append(file)
                dataSource = rmtPth + file
                dataDestination = locPth + file
                try:
                    connection.get(dataSource, dataDestination)
                except:
                    pass
            elif Previous_Date_7.date() <= filedate.date() <= Previous_Date.date() and splitFileType =='AROpenItemDaily':
                listDownloadFileAR.append(file)
                dataSource = rmtPth + file
                dataDestination = locPth + file
                try:
                    connection.get(dataSource, dataDestination)
                    listDownloadFileAR.append(file)
                except:
                    pass
        if len(listDownloadFileAR) == 0:
            print(f"Total downloaded file:- {listDownloadFileAR} ")

        return listDownloadFileAR

In [210]:
def AR_combine_last7Days(locPth):
    print(f"LocPath:- {locPth}")
    Current_Date = datetime.datetime.today()
    pathtoSave = locPth + "CA_AROpenItemsWeeklyCombined_" + str(Current_Date.date()) + ".txt"
    openArConsolidatedlist = [ ]

    for fil in os.listdir(locPth):
        if fil.split('_')[1] == 'AROpenItemDaily':
            strdate = fil.split('_')[2][:8]
            m = int(strdate[:2])
            d = int(strdate[2:4])
            y = int(strdate[4:])
            filedate = datetime.datetime(y, m, d).date()
            
            print(f'locPth+fil :- {locPth+fil}')
            aropen = pd.read_csv(locPth+fil, sep="|", encoding= 'unicode_escape')
            
            aropen['fileDate'] = filedate
            col = ['DocumentNo', 'Customer', 'CompCode', 'FiscalYear', 'Doc.Date', 'ItemNo']
            aropen['Combined'] = aropen[col].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
            aropen = aropen.drop_duplicates(subset=['Combined'])
            openArConsolidatedlist.append(aropen)

    combinedDF = pd.concat(openArConsolidatedlist)
    finalDF = combinedDF.sort_values('fileDate').drop_duplicates('Combined', keep='last')
    FinalDF_us10 = finalDF[finalDF['CompCode'] == 'US10']
    FinalDF_us10.to_csv(pathtoSave,sep="|", index = False)

In [211]:
# AR_combine_last7Days(locPth + 'Ar_Decrypted/')

# Upload file to Archinve folder

In [212]:
def uploadToArchived(locPth, rmtPth):  #upload all files from local archived to remote archived
    for de in os.listdir(locPth):
        print(f"de:-{de}")
        print(f"locPth_de:- {locPth + de} , rmtPth_de:- {rmtPth + de}")
        connection.put(locPth + de , rmtPth + de)
        print(f"Uploaded file to remote archive folder {rmtPth + de}")

# Decrypt and save in local folder

In [213]:
def decryptAndSaveLocal(locPth ,pathLocalToSave):
    decrySuccess = []
    decryNotSuccess = []
    for fil in os.listdir(locPth):
        if "TEST" not in fil:
            pathtoSave = pathLocalToSave + fil.split('.')[0]+".txt" #path to save encrypted file
            df = pd.DataFrame()
            fil = locPth + fil
            try:
                df = decryptFile(fil)
                print(f"fil:-{fil}")
                df.to_csv(pathtoSave, sep='|',error_bad_lines=False,index = False)
                decrySuccess.append(fil)
                print(f"Decrypted file and saved in local folder name:- {fil} ")
            except:
                df.to_csv(pathtoSave, sep='|', encoding='cp1252', index = False)
                print(f"Decrypted file and saved in local folder name:- {fil} ")
                decryNotSuccess.append(fil)

# Upload decrytpted file back in SFTP server

In [214]:
def uploadDecryptedfile(locPth, rmtPth ): #upload decrypted file
    for de in os.listdir(locPth):
        lopath = locPth+de
        destpath = rmtPth+de
        connection.put(lopath, destpath)
    print(f"upload decrypted file complete :-{rmtPth}")

In [215]:
# locPth
# for file in os.listdir(locPth + 'Ar_Archive/'):
#     print(file)

In [216]:
# locPth
# for file in os.listdir(locPth + 'Ar_Decrypted/'):
#     print(file. )

In [217]:
# f = open(pafile + "CA_AROpenItemDaily_03312023013130.txt", "r")
# print(f.read()) 


In [218]:
# df = decryptFile('/home/wsuser/work/Ar_Archive/CA_CustMstrDaily_04022023013005.txt.gpg')

In [219]:
pathLocalToSaveAR = locPth + 'Ar_Decrypted/'
decryptAndSaveLocal(locPth + 'Ar_Archive/', pathLocalToSaveAR)

In [220]:
currentDate_AR = datetime.datetime.today()
currentDate_AR_Day = currentDate_AR.strftime('%A')
if currentDate_AR_Day == "Monday":   # execution on Monday
    nDays = 1
    previousDay = 7
    
if currentDate_AR_Day == "Tuesday":   # execution on Monday
    nDays = 2
    previousDay = 7

if currentDate_AR_Day == "Wednesday": # execution on Wednesday
    nDays = 0
    previousDay = 9

if currentDate_AR_Day == "Thursday": # execution on Wednesday
    nDays = 1
    previousDay = 10

if currentDate_AR_Day == "Friday": # execution on Wednesday
    nDays = 2
    previousDay = 4
# ARrmtPath = '/ANALYTICS/PROD/'
ARrmtPath = '/ANALYTICS/PROCESSED_TEST/'
listDownloadfileAR = download_AR(ARrmtPath, locPth + 'Ar_Archive/',nDays,previousDay)  # download AR data
if len(listDownloadfileAR) != 0:
    print(f"1 AP data download complete, List of downloaded file:-{listDownloadfileAR}")

    uploadToArchived(locPth=locPth + 'Ar_Archive/', rmtPth=rmtPth + 'Ar_Archive/')  # upload all files to remote
    print("2 AR files uploaded on SFTP server archive folder")

   
    pathLocalToSaveAR = locPth + 'Ar_Decrypted/'
    decryptAndSaveLocal(locPth + 'Ar_Archive/', pathLocalToSaveAR)
    print("3 AR file decrypted and saved in local folder")
    
    AR_combine_last7Days(locPth + 'Ar_Decrypted/')
    print("AR file combined")
    

    uploadDecryptedfile(locPth + 'Ar_Decrypted/', rmtPth + 'Ar_Decrypted/')     #
    print("4 AR decrypted file uploaded on SFTP server")

1 AP data download complete, List of downloaded file:-['CA_AROpenItemDaily_03312023013130.txt.gpg', 'CA_AROpenItemDaily_03312023013130.txt.gpg', 'CA_ARDisputesDaily_04022023013617.txt.gpg', 'CA_AROpenItemDaily_03302023013059.txt.gpg', 'CA_AROpenItemDaily_03302023013059.txt.gpg', 'CA_AROpenItemDaily_04022023013122.txt.gpg', 'CA_ARClosedItemDaily_04022023013216.txt.gpg', 'CA_CustMstrDaily_04022023013005.txt.gpg', 'CA_AROpenItemDaily_03292023013048.txt.gpg', 'CA_AROpenItemDaily_03292023013048.txt.gpg', 'CA_AROpenItemDaily_03282023013056.txt.gpg', 'CA_AROpenItemDaily_03282023013056.txt.gpg', 'CA_AROpenItemDaily_04012023013117.txt.gpg', 'CA_AROpenItemDaily_04012023013117.txt.gpg']
de:-CA_CustMstrDaily_04022023013005.txt.gpg
locPth_de:- /home/wsuser/work/Ar_Archive/CA_CustMstrDaily_04022023013005.txt.gpg , rmtPth_de:- /ANALYTICS/WCO/Ar_Archive/CA_CustMstrDaily_04022023013005.txt.gpg
Uploaded file to remote archive folder /ANALYTICS/WCO/Ar_Archive/CA_CustMstrDaily_04022023013005.txt.gpg
de:-C

/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/pgpy/constants.py:192: CryptographyDeprecationWarning: IDEA has been deprecated
  bs = {SymmetricKeyAlgorithm.IDEA: algorithms.IDEA,
/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/pgpy/constants.py:194: CryptographyDeprecationWarning: CAST5 has been deprecated
  SymmetricKeyAlgorithm.CAST5: algorithms.CAST5,
/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/pgpy/constants.py:195: CryptographyDeprecationWarning: Blowfish has been deprecated
  SymmetricKeyAlgorithm.Blowfish: algorithms.Blowfish,
/tmp/wsuser/ipykernel_233/3479021364.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(BytesIO(d), error_bad_lines=False, sep='|')
/tmp/wsuser/ipykernel_233/3479021364.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.



fil:-/home/wsuser/work/Ar_Archive/CA_CustMstrDaily_04022023013005.txt.gpg
Decrypted file and saved in local folder name:- /home/wsuser/work/Ar_Archive/CA_CustMstrDaily_04022023013005.txt.gpg 


/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/pgpy/constants.py:192: CryptographyDeprecationWarning: IDEA has been deprecated
  bs = {SymmetricKeyAlgorithm.IDEA: algorithms.IDEA,
/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/pgpy/constants.py:194: CryptographyDeprecationWarning: CAST5 has been deprecated
  SymmetricKeyAlgorithm.CAST5: algorithms.CAST5,
/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/pgpy/constants.py:195: CryptographyDeprecationWarning: Blowfish has been deprecated
  SymmetricKeyAlgorithm.Blowfish: algorithms.Blowfish,
/tmp/wsuser/ipykernel_233/3479021364.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(BytesIO(d), error_bad_lines=False, sep='|')
/tmp/wsuser/ipykernel_233/3479021364.py:9: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(BytesIO(d), error_b

fil:-/home/wsuser/work/Ar_Archive/CA_AROpenItemDaily_03312023013130.txt.gpg
Decrypted file and saved in local folder name:- /home/wsuser/work/Ar_Archive/CA_AROpenItemDaily_03312023013130.txt.gpg 


...

locPth+fil :- /home/wsuser/work/Ar_Decrypted/CA_AROpenItemDaily_03302023013059.txt
locPth+fil :- /home/wsuser/work/Ar_Decrypted/CA_AROpenItemDaily_04012023013117.txt


/tmp/wsuser/ipykernel_233/3918535104.py:16: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  aropen = pd.read_csv(locPth+fil, sep="|", encoding= 'unicode_escape')


locPth+fil :- /home/wsuser/work/Ar_Decrypted/CA_AROpenItemDaily_03292023013048.txt
locPth+fil :- /home/wsuser/work/Ar_Decrypted/CA_AROpenItemDaily_04022023013122.txt
locPth+fil :- /home/wsuser/work/Ar_Decrypted/CA_AROpenItemDaily_03282023013056.txt


/tmp/wsuser/ipykernel_233/3918535104.py:16: DtypeWarning: Columns (8,36) have mixed types. Specify dtype option on import or set low_memory=False.
  aropen = pd.read_csv(locPth+fil, sep="|", encoding= 'unicode_escape')


AR file combined
upload decrypted file complete :-/ANALYTICS/WCO/Ar_Decrypted/
4 AR decrypted file uploaded on SFTP server
